In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
from jpnotebooks.Sessions.cfg_merge import *
import os
import lxml.etree
import json
import csv
import collections

In [37]:
class ConfigFile():
    # supported file types for built in quick parsing 
    
    load_type = "lines"
    # load_type = "xml"
    # load_type = "csv"
    # load_type = "json"
    
    # Hello Version the file belongs to
    version = "3.0.8"
    
    load_other = None
    
    def __init__(self, ob=None):
        self.load_other = self.load_other or {}
        if ob is not None:
            self.load(ob)
            
    def add_loader(self, typ, func):
        self.load_other[typ] = func
        
    def load(self, ob):
        # arbitrary load function
        if isinstance(ob, str):
            if os.path.exists(ob):
                self.load_fn(ob)
            else:
                # got file content body as text i hope...
                self.load_text(ob)
        elif hasattr(fp, 'read'):
            self.load_fp(fp)
        elif isinstance(ob, (list, tuple)):
            # list of lines ??
            self.load_lines(lines)
        elif isinstance(ob, dict):
            self.load_lines(list(ob.keys()))
        elif typ(ob) in self.load_other:
            self.load_other[type(ob)](ob)
        else:
            raise TypeError("Don't know how to load type '%s'"%ob.__class__.__name__)
            
    def _load_check(self, typ):
        if self.load_type != typ:
            raise TypeError("Config type '%s' can't load from '%s'"%self.__class__.__name__, typ) 
        
    def load_text(self, text):
        if self.typ == "lines":
            self.load_lines(text.splitlines())
        elif self.typ == "xml":
            self.load_xml(lxml.etree.XML(text))
        elif self.typ == "json":
            self.load_xml(json.loads(text))
        elif self.typ == "csv":
            self.load_csv(list(csv.reader(text)))
        else:
            raise TypeError(self.typ)
        
    def load_fd(self, fd):
        if fd.mode not in ("r", "rb", "br"):
            raise ValueError("File wasn't in read mode")
        if self.typ == "xml" and "b" in mode:  # builtin xml must be binary, not sure about lxml
            self.load_xml(lxml.etree.parse(fd))
        elif self.typ == "json":
            self.load_json(json.load(fd))
        else:
            self.load_text(fd.read())
        
    def load_fn(self, fn):
        with open(fn, 'r') as fd:
            self.load_fd(fd)
        
    def load_lines(self, lines):
        raise NotImplementedError()
    
    def load_xml(self, xml):
        raise NotImplementedError()
    
    def load_csv(self, text):
        raise NotImplementedError()
    
    def load_json(self, json):
        raise NotImplementedError()
        
    def load_data(self, data):
        self.data = data.copy()
        
    def copy(self):
        f = self.__class__()
        f.load_data(self.data)
        return f
    
class _ClassList():
    def __init__(self):
        self.l = []
        self.d = {}
    
    def add(self, v, c):
        if v in self.d:
            self.remove(v)
        self.d[v] = c
        self.l.append((v,c))
        self.sort()
        
    def remove(self, v):
        del self.d[v]
        for i, (vers,c) in enumerate(self.l):
            if v == vers:
                break
        if v == vers:  # verify break
            del self.l[i]
            
    def __iter__(self):
        return iter(self.l)

    def get(self, key):
        return self.d.get(key)
            
    def sort(self):
        # since this is always called on append, 
        # we only ever have to sort if v[-1] is less than v[-2]
        if len(self.l) < 2:
            return
        v1 = self.l[-1][0]
        v2 = self.l[-2][0]
        if v1 < v2:
            self.l.sort(key=lambda t:t[0].split("."))
    
# alarm types
_typs = "alarms", "logger settings", "system variables", "email settings", "cal factors"

def _t2s(t):
    t = t.lower()
    r = t.rfind(".")
    if r != -1:
        t = t[:r]
    if t in ("alarms on", "alarms off", "alarm"):
        t = "alarms"
    elif t in ("logging on", "logging off"):
        t = "logger settings"
    elif t == "email alert settings":
        t = "email settings"
    return t

_class_reg = {_t:_ClassList() for _t in _typs}
def file_class(typ, version):
    s = _t2s(typ)
    cl = _class_reg.get(s)
    if cl is None:
        raise ValueError("Unrecognized file type")
    c = None
    for v, c in cl:
        if v == version:
            return c
    if c is None:
        raise ValueError("Uh oh! empty list")
    log2("Failed to find version match for v%s, fall back to v%s"%(version, v))
    return c 

def _register3(klass, version, typ):
    _class_reg[typ].add(version, klass)
    
def _register2(klass, typ):
    _register3(klass, klass.version, typ)
    
def _register(klass):
    _register2(klass, klass.typ)
    
def _pathsearch(nodes, current, target, path):
    for start, dst in nodes:
        if start == current:
            if dst == target:
                return True
            path.append(dst)
            if _pathsearch(nodes, dst, target, path):
                return True
            path.pop()
    return False
            
def convert(file, version):
    if file.version == version:
        return file.copy()
    path = []
    
    
class AlarmsFile():
    typ = "alarms"
    def tostr(self):
        b = []
        for n, (a, b) in self.data.items():
            b.append(",".join((n, str(a), str(b))))
        return "\n".join(b)
        
class AlarmsFile1(AlarmsFile):
    version = "1.0.0"
    def load_lines(self, lines):
        data = OrderedDict()
        assert lines[0] == "name, audable, ignore", lines[0]  # sic
        for line in lines[1:]:
            n, a, i = line.split(",", 1)
            data[name] = int(a), int(i)
        self.data = data
            
class AlarmsFile2(AlarmsFile):
    version = "2.0.0"
    def load_lines(self, lines):
        data = OrderedDict()
        assert lines[0] == "name, audible, ignore", lines[0]  # sic
        for line in lines[1:]:
            n, a, i = line.split(",", 1)
            data[name] = int(a), int(i)
        self.data = data
            
class AlarmsFile3(AlarmsFile):
    version = "3.0.0"
    def load_lines(s):
        assert lines[0] == "name, notify, audible", lines[0]  # sic
        for line in lines[1:]:
            n, a, i = line.split(",", 1)
            data[name] = int(a), int(i)
        self.data = data
            
_register(AlarmsFile1)
_register(AlarmsFile2)
_register(AlarmsFile3)

def _cnv_alm_12_3(af):
    data = OrderedDict()
    for name, (a, i) in af.data.items():
        data[name] = 1-i, a
    rv = AlarmsFile3()
    rv.load_data(data)
    return rv
\
_alm_conv = [
    ("1.0.0", "2.0.0", _cnv_alm_12_3),
    ("1.0.0", "3.0.0", _cnv_alm_12_3),
    ("2.0.0", "3.0.0", _cnv_alm_12_3),
]

In [ ]:
nodes = [
    (1, 2),
    (2, 3), 
    (4, 5),
    (3, 5),
    (4, 12),
    (5, 7),
    (7, 12)
]
_pathsearch(nodes)